In [20]:
# Now universal MACE finetuning on T2
#!/usr/bin/env python3
import os
import subprocess
import sys

def main():
    # ——— Environment setup ———
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

    cmd = [
        "mace_run_train",
        # === General settings ===
        "--name",              "mace_T2_including_replay_w2",
        "--model",             "MACE",
        "--num_interactions",  "2",
        "--foundation_model",  "/home/phanim/harshitrawat/summer/mace_models/universal/2024-01-07-mace-128-L2_epoch-199.model",
        "--foundation_model_readout",
    # --- MP replay (pretraining head) ---
        "--pt_train_file","/home/phanim/harshitrawat/summer/replay_data/mp_finetuning-mace_T2_mp_replay_run-42.xyz",              # <- MP replay shortcut
        "--atomic_numbers","[3,8,40,57]",    # Li, O, Zr, La
        "--multiheads_finetuning","True",

        "--train_file","/home/phanim/harshitrawat/summer/iteration_3-Copy1/one.xyz",
        "--valid_fraction", "0.5",
        "--batch_size",        "1",
        "--valid_batch_size",  "1",

        "--device",            "cpu",

        # === Loss function weights ===
        "--forces_weight",     "0",         # Increased force weight to balance energy better
        "--energy_weight",     "0",   
        "--stress_weight", "0",             # Reduced from 100 → avoid dominance + stabilize energy RMSE

        # === Learning setup ===
        "--lr",                "0.006",      # Explicit learning rate (0.0001 is too low → stagnation)
        "--scheduler_patience","4",          # Reduce LR if val loss doesn’t improve in 3 epochs
        "--clip_grad",         "1",        # Avoid exploding gradients — essential when energy_weight is high
        "--weight_decay",      "1e-8",       # Mild regularization to prevent overfitting

        # === EMA helps smooth loss curve ===
        #"--ema_decay",         "0.999",     # Smooths validation loss and helps final convergence

        # === Domain + training settings ===
        "--r_max",             "5.0",
        "--max_num_epochs",    "1",
        "--E0s",               "average",    # Still allowed — could optionally be replaced by manual E0s
        "--seed",              "84",
        "--patience",     "8",

        "--restart_latest",                   # Resumes from checkpoint if available
    ]

    print("Running:", " \\\n    ".join(cmd), file=sys.stderr)
    subprocess.run(cmd, check=True)

if __name__ == "__main__":
    main()


Running: mace_run_train \
    --name \
    mace_T2_including_replay_w2 \
    --model \
    MACE \
    --num_interactions \
    2 \
    --foundation_model \
    /home/phanim/harshitrawat/summer/mace_models/universal/2024-01-07-mace-128-L2_epoch-199.model \
    --foundation_model_readout \
    --train_file \
    /home/phanim/harshitrawat/summer/iteration_3-Copy1/one.xyz \
    --valid_fraction \
    0.5 \
    --batch_size \
    1 \
    --valid_batch_size \
    1 \
    --device \
    cpu \
    --forces_weight \
    0 \
    --energy_weight \
    0 \
    --stress_weight \
    0 \
    --lr \
    0.006 \
    --scheduler_patience \
    4 \
    --clip_grad \
    1 \
    --weight_decay \
    1e-8 \
    --r_max \
    5.0 \
    --max_num_epochs \
    1 \
    --E0s \
    {3: -1.2302615750354944} \
    --seed \
    84 \
    --patience \
    8 \
    --restart_latest
/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/e3nn/o3/_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO

2025-08-21 20:19:52.743 INFO: ===========VERIFYING SETTINGS===========
2025-08-21 20:19:52.743 INFO: MACE version: 0.3.14
2025-08-21 20:19:52.743 INFO: Using CPU
2025-08-21 20:19:52.864 INFO: Using foundation model /home/phanim/harshitrawat/summer/mace_models/universal/2024-01-07-mace-128-L2_epoch-199.model as initial checkpoint.
2025-08-21 20:19:52.864 WARNING: Using multiheads finetuning with a foundation model that is not a Materials Project model, need to provied a path to a pretraining file with --pt_train_file.
2025-08-21 20:19:52.864 INFO: ===========LOADING INPUT DATA===========
2025-08-21 20:19:52.864 INFO: Using heads: ['Default']
2025-08-21 20:19:52.864 INFO: Using the key specifications to parse data:
2025-08-21 20:19:52.864 INFO: Default: KeySpecification(info_keys={'energy': 'REF_energy', 'stress': 'REF_stress', 'virials': 'REF_virials', 'dipole': 'dipole', 'head': 'head', 'elec_temp': 'elec_temp', 'total_charge': 'total_charge', 'polarizability': 'polarizability', 'total

/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/mace/cli/run_train.py:152: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  model_foundation = torch.load(


2025-08-21 20:19:53.175 INFO: Average number of neighbors: nan
2025-08-21 20:19:53.175 INFO: During training the following quantities will be reported: energy, forces
2025-08-21 20:19:53.175 INFO: ===========MODEL DETAILS===========
2025-08-21 20:19:53.177 WARNING: Standard deviation of the scaling is zero, Changing to no scaling
2025-08-21 20:19:53.177 INFO: Loading FOUNDATION model
2025-08-21 20:19:53.177 INFO: Using filtered elements: [3]
2025-08-21 20:19:53.177 INFO: Model configuration extracted from foundation model
2025-08-21 20:19:53.177 INFO: Using weighted loss function for fine-tuning
2025-08-21 20:19:53.177 INFO: Message passing with hidden irreps 128x0e+128x1o+128x2e)
2025-08-21 20:19:53.177 INFO: 2 layers, each with correlation order: 3 (body order: 4) and spherical harmonics up to: l=3
2025-08-21 20:19:53.177 INFO: Radial cutoff: 6.0 A (total receptive field for each atom: 12.0 A)
2025-08-21 20:19:53.177 INFO: Distance transform for radial basis functions: None


/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warn

2025-08-21 20:20:04.057 INFO: Total number of parameters: 723866
2025-08-21 20:20:04.057 INFO: 
2025-08-21 20:20:04.057 INFO: ===========OPTIMIZER INFORMATION===========
2025-08-21 20:20:04.057 INFO: Using ADAM as parameter optimizer
2025-08-21 20:20:04.057 INFO: Batch size: 1
2025-08-21 20:20:04.057 INFO: Number of gradient updates: 1
2025-08-21 20:20:04.057 INFO: Learning rate: 0.006, weight decay: 1e-08
2025-08-21 20:20:04.057 INFO: WeightedEnergyForcesLoss(energy_weight=0.000, forces_weight=0.000)
2025-08-21 20:20:04.059 WARNING: No SWA checkpoint found, while SWA is enabled. Compare the swa_start parameter and the latest checkpoint.
2025-08-21 20:20:04.059 INFO: Loading checkpoint: ./checkpoints/mace_T2_including_replay_w2_run-84_epoch-77.pt
2025-08-21 20:20:04.213 INFO: Using gradient clipping with tolerance=1.000
2025-08-21 20:20:04.213 INFO: 
2025-08-21 20:20:04.213 INFO: ===========TRAINING===========
2025-08-21 20:20:04.213 INFO: Started training, reporting errors on validati

Traceback (most recent call last):
  File "/home/phanim/harshitrawat/miniconda3/bin/mace_run_train", line 8, in <module>
    sys.exit(main())
             ~~~~^^
  File "/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/mace/cli/run_train.py", line 77, in main
    run(args)
    ~~~^^^^^^
  File "/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/mace/cli/run_train.py", line 940, in run
    epoch = checkpoint_handler.load_latest(
        state=tools.CheckpointState(model, optimizer, lr_scheduler),
        swa=swa_eval,
        device=device,
    )
  File "/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/mace/tools/checkpoint.py", line 215, in load_latest
    self.builder.load_checkpoint(state=state, checkpoint=checkpoint, strict=strict)
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/mace/tools/checkpoint.py", line 40, in load_chec

CalledProcessError: Command '['mace_run_train', '--name', 'mace_T2_including_replay_w2', '--model', 'MACE', '--num_interactions', '2', '--foundation_model', '/home/phanim/harshitrawat/summer/mace_models/universal/2024-01-07-mace-128-L2_epoch-199.model', '--foundation_model_readout', '--train_file', '/home/phanim/harshitrawat/summer/iteration_3-Copy1/one.xyz', '--valid_fraction', '0.5', '--batch_size', '1', '--valid_batch_size', '1', '--device', 'cpu', '--forces_weight', '0', '--energy_weight', '0', '--stress_weight', '0', '--lr', '0.006', '--scheduler_patience', '4', '--clip_grad', '1', '--weight_decay', '1e-8', '--r_max', '5.0', '--max_num_epochs', '1', '--E0s', '{3: -1.2302615750354944}', '--seed', '84', '--patience', '8', '--restart_latest']' returned non-zero exit status 1.

In [6]:
# Now universal MACE finetuning on T2
#!/usr/bin/env python3
import os
import subprocess
import sys

def main():
    # ——— Environment setup ———
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

    cmd = [
        "mace_run_train",
        # === General settings ===
        "--name",              "template_mace_T2_including_replay_w2",
        "--model",             "MACE",
        "--num_interactions",  "2",
        "--foundation_model",  "/home/phanim/harshitrawat/summer/mace_models/universal/2024-01-07-mace-128-L2_epoch-199.model",
        "--foundation_model_readout",
    # --- MP replay (pretraining head) ---
        # "--pt_train_file","/home/phanim/harshitrawat/summer/replay_data/mp_finetuning-mace_T2_mp_replay_run-42.xyz",              # <- MP replay shortcut
        # "--atomic_numbers","[3,8,40,57]",    # Li, O, Zr, La
        # "--multiheads_finetuning","True",

        "--train_file","/home/phanim/harshitrawat/summer/iteration_3-Copy1/one.xyz",
        "--valid_fraction", "0.5",
        "--batch_size",        "1",
        "--valid_batch_size",  "1",

        "--device",            "cpu",

        # === Loss function weights ===
        "--forces_weight",     "0",         # Increased force weight to balance energy better
        "--energy_weight",     "10",   
        "--stress_weight", "0",             # Reduced from 100 → avoid dominance + stabilize energy RMSE

        # === Learning setup ===
        "--lr",                "0.006",      # Explicit learning rate (0.0001 is too low → stagnation)
        "--scheduler_patience","4",          # Reduce LR if val loss doesn’t improve in 3 epochs
        "--clip_grad",         "1",        # Avoid exploding gradients — essential when energy_weight is high
        "--weight_decay",      "1e-8",       # Mild regularization to prevent overfitting

        # === EMA helps smooth loss curve ===
        #"--ema_decay",         "0.999",     # Smooths validation loss and helps final convergence

        # === Domain + training settings ===
        "--r_max",             "5.0",
        "--max_num_epochs",    "1",
        "--E0s",               "average",    # Still allowed — could optionally be replaced by manual E0s
        "--seed",              "84",
        "--patience",     "8",

        "--restart_latest",                   # Resumes from checkpoint if available
    ]

    print("Running:", " \\\n    ".join(cmd), file=sys.stderr)
    subprocess.run(cmd, check=True)

if __name__ == "__main__":
    main()


✅ Keys renamed to REF_* and saved to:
/home/phanim/harshitrawat/summer/replay_data/mp_finetuning-mace_T2_mp_replay_run-42_REFkeys.xyz


In [3]:
# Now universal MACE finetuning on T2
#!/usr/bin/env python3
import os
import subprocess
import sys

def main():
    # ——— Environment setup ———
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

    cmd = [
        "mace_run_train",
        # === General settings ===
        "--name",              "dummy_mace_T2_including_replay_w2",
        "--model",             "MACE",
        "--num_interactions",  "2",
        "--foundation_model",  "/home/phanim/harshitrawat/summer/mace_models/universal/2024-01-07-mace-128-L2_epoch-199.model",
        "--foundation_model_readout",
    # --- MP replay (pretraining head) ---
        "--pt_train_file","/home/phanim/harshitrawat/summer/replay_data/mp_finetuning-mace_T2_mp_replay_run-42.xyz",              # <- MP replay shortcut
        "--atomic_numbers","[3,8,40,57]",    # Li, O, Zr, La
        "--multiheads_finetuning","True",

        "--train_file", "/home/phanim/harshitrawat/summer/dummy.extxyz",
        "--valid_file", "/home/phanim/harshitrawat/summer/dummy.extxyz",

        "--batch_size",        "2",
        "--valid_batch_size",  "1",

        "--device",            "cuda",

        # === Loss function weights ===
        "--forces_weight",     "0",         # Increased force weight to balance energy better
        "--energy_weight",     "10",   
        "--stress_weight", "0",             # Reduced from 100 → avoid dominance + stabilize energy RMSE

        # === Learning setup ===
        "--lr",                "0.006",      # Explicit learning rate (0.0001 is too low → stagnation)
        "--scheduler_patience","4",          # Reduce LR if val loss doesn’t improve in 3 epochs
        "--clip_grad",         "1",        # Avoid exploding gradients — essential when energy_weight is high
        "--weight_decay",      "1e-8",       # Mild regularization to prevent overfitting

        # === EMA helps smooth loss curve ===
        #"--ema_decay",         "0.999",     # Smooths validation loss and helps final convergence

        # === Domain + training settings ===
        "--r_max",             "5.0",
        "--max_num_epochs",    "1",
        "--E0s",               "{3: -1.2302615750354944, 8: -23.049110738413006, 40: 23.367646191010394, 57: 15.192898072498549}",    # Still allowed — could optionally be replaced by manual E0s
        "--seed",              "84",
        "--patience",     "8",

        "--restart_latest",                   # Resumes from checkpoint if available
    ]

    print("Running:", " \\\n    ".join(cmd), file=sys.stderr)
    subprocess.run(cmd, check=True)

if __name__ == "__main__":
    main()


Running: mace_run_train \
    --name \
    dummy_mace_T2_including_replay_w2 \
    --model \
    MACE \
    --num_interactions \
    2 \
    --foundation_model \
    /home/phanim/harshitrawat/summer/mace_models/universal/2024-01-07-mace-128-L2_epoch-199.model \
    --foundation_model_readout \
    --pt_train_file \
    /home/phanim/harshitrawat/summer/replay_data/mp_finetuning-mace_T2_mp_replay_run-42.xyz \
    --atomic_numbers \
    [3,8,40,57] \
    --multiheads_finetuning \
    True \
    --train_file \
    /home/phanim/harshitrawat/summer/dummy.extxyz \
    --valid_file \
    /home/phanim/harshitrawat/summer/dummy.extxyz \
    --batch_size \
    2 \
    --valid_batch_size \
    1 \
    --device \
    cuda \
    --forces_weight \
    0 \
    --energy_weight \
    10 \
    --stress_weight \
    0 \
    --lr \
    0.006 \
    --scheduler_patience \
    4 \
    --clip_grad \
    1 \
    --weight_decay \
    1e-8 \
    --r_max \
    5.0 \
    --max_num_epochs \
    1 \
    --E0s \
    {

2025-08-22 03:17:46.609 INFO: ===========VERIFYING SETTINGS===========
2025-08-22 03:17:46.609 INFO: MACE version: 0.3.14
2025-08-22 03:17:46.655 INFO: CUDA version: 12.6, CUDA device: 0


/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/mace/cli/run_train.py:152: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  model_foundation = torch.load(


2025-08-22 03:17:47.240 INFO: Using foundation model /home/phanim/harshitrawat/summer/mace_models/universal/2024-01-07-mace-128-L2_epoch-199.model as initial checkpoint.
2025-08-22 03:17:47.241 INFO: Multihead finetuning mode, setting learning rate to 0.0001 and EMA to True. To use a different learning rate, set --force_mh_ft_lr=True.
2025-08-22 03:17:47.241 INFO: Using multiheads finetuning mode, setting learning rate to 0.0001 and EMA to True
2025-08-22 03:17:47.241 INFO: ===========LOADING INPUT DATA===========
2025-08-22 03:17:47.241 INFO: Using heads: ['Default', 'pt_head']
2025-08-22 03:17:47.241 INFO: Using the key specifications to parse data:
2025-08-22 03:17:47.241 INFO: Default: KeySpecification(info_keys={'energy': 'REF_energy', 'stress': 'REF_stress', 'virials': 'REF_virials', 'dipole': 'dipole', 'head': 'head', 'elec_temp': 'elec_temp', 'total_charge': 'total_charge', 'polarizability': 'polarizability', 'total_spin': 'total_spin'}, arrays_keys={'forces': 'REF_forces', 'ch

/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(
/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/torch/jit/_check.py:178: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warn

2025-08-22 03:18:13.041 INFO: Total number of parameters: 5556810
2025-08-22 03:18:13.042 INFO: 
2025-08-22 03:18:13.042 INFO: ===========OPTIMIZER INFORMATION===========
2025-08-22 03:18:13.042 INFO: Using ADAM as parameter optimizer
2025-08-22 03:18:13.042 INFO: Batch size: 2
2025-08-22 03:18:13.042 INFO: Using Exponential Moving Average with decay: 0.99999
2025-08-22 03:18:13.042 INFO: Number of gradient updates: 4951
2025-08-22 03:18:13.042 INFO: Learning rate: 0.0001, weight decay: 1e-08
2025-08-22 03:18:13.042 INFO: UniversalLoss(energy_weight=10.000, forces_weight=0.000, stress_weight=0.000)
2025-08-22 03:18:13.046 WARNING: Cannot find checkpoint with tag 'dummy_mace_T2_including_replay_w2_run-84' in './checkpoints'
2025-08-22 03:18:13.047 INFO: Using gradient clipping with tolerance=1.000
2025-08-22 03:18:13.047 INFO: 
2025-08-22 03:18:13.047 INFO: ===========TRAINING===========
2025-08-22 03:18:13.047 INFO: Started training, reporting errors on validation set
2025-08-22 03:18:

/home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages/mace/tools/checkpoint.py:187: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=checkpoint_info.path, map_location=device),


2025-08-22 03:39:40.823 INFO: Loaded Stage one model from epoch 0 for evaluation
2025-08-22 03:39:40.823 INFO: Saving model to checkpoints/dummy_mace_T2_including_replay_w2_run-84.model
2025-08-22 03:39:41.158 INFO: Compiling model, saving metadata to dummy_mace_T2_including_replay_w2_compiled.model
2025-08-22 03:39:41.776 INFO: Computing metrics for training, validation, and test sets
2025-08-22 03:39:41.777 INFO: Skipping evaluation for heads: ['pt_head']
2025-08-22 03:39:41.777 INFO: Evaluating train_Default ...
2025-08-22 03:40:31.165 INFO: Skipping evaluation of train_pt_head (in skip_heads list)
2025-08-22 03:40:31.165 INFO: Evaluating valid_Default ...
2025-08-22 03:40:31.366 INFO: Skipping evaluation of valid_pt_head (in skip_heads list)
2025-08-22 03:40:31.374 INFO: Error-table on TRAIN and VALID:
+---------------+---------------------+------------------+-------------------+---------------------------------------+
|  config_type  | RMSE E / meV / atom | RMSE F / meV / A | rela

In [5]:
import torch

ckpt_path = "/home/phanim/harshitrawat/summer/iteration_3/checkpoints/mace_T2_including_replay_w2_run-84_epoch-77.pt"
template_model_path = "/home/phanim/harshitrawat/summer/iteration_3-Copy1/checkpoints/dummy_mace_T2_including_replay_w2_run-84.model"
out_model_path = "/home/phanim/harshitrawat/summer/formation_energy/mace_T2_frozen_220825.model"

# Load both with full objects (not weights_only)
ckpt  = torch.load(ckpt_path, map_location="cpu", weights_only=False)
model = torch.load(template_model_path, map_location="cpu", weights_only=False)

# Inject weights from checkpoint
missing, unexpected = model.load_state_dict(ckpt["model"], strict=False)
print("missing keys:", missing)
print("unexpected keys:", unexpected)

# Save the usable .model
torch.save(model, out_model_path)
print("✅ Wrote:", out_model_path)


missing keys: []
unexpected keys: []
✅ Wrote: /home/phanim/harshitrawat/summer/formation_energy/mace_T2_frozen_220825.model


In [6]:
from mace.calculators import MACECalculator
mace_calc = MACECalculator(model_paths=["/home/phanim/harshitrawat/summer/formation_energy/mace_T2_frozen_220825.model"], device="cuda")  # or "cpu"
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.core import Structure
adaptor = AseAtomsAdaptor()

pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/Li.cif")  # e.g. for Li
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Li = total_energy / len(ase_atoms)
print(f"Li: μ_model = {mu_model_Li:.6f} eV/atom")
# Let us do this for La, Zr, and O as well
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/La.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_La = total_energy / len(ase_atoms)
print(f"La: μ_model = {mu_model_La:.6f} eV/atom")
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/Zr.cif")
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_Zr = total_energy / len(ase_atoms)
print(f"Zr: μ_model = {mu_model_Zr:.6f} eV/atom")
pmg_structure = Structure.from_file("/home/phanim/harshitrawat/summer/formation_energy/cifs/O2.cif")  # Needs to be a periodic solid O2 structure
ase_atoms = adaptor.get_atoms(pmg_structure)
ase_atoms.calc = mace_calc
total_energy = ase_atoms.get_potential_energy()
mu_model_O = total_energy / len(ase_atoms)
print(f"O: μ_model = {mu_model_O:.6f} eV/atom")

/home/phanim/harshitrawat/mace/mace/calculators/mace.py:166: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['pt_head', 'Default']
No dtype selected, switching to float64 to match model dtype.
Li: μ_model = -1.738880 eV/atom


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 4 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


La: μ_model = 18.890417 eV/atom
Zr: μ_model = 26.440122 eV/atom
O: μ_model = -20.875662 eV/atom


In [9]:
import os
# ---- 1. Load the MACE model -------------------------------------------
calculator = MACECalculator(model_paths=["/home/phanim/harshitrawat/summer/formation_energy/mace_T2_frozen_220825.model"], device="cuda")  # or "cpu"

# ---- 2. Reference μ_model from MACE -----------------------------------

mu_mace = {
    "Li": -1.738880,
    "La": 18.890417,
    "Zr": 26.440122,
    "O":  -20.875662,
}

# ---- 3. CIF files ------------------------------------------------------
cif_dir = "/home/phanim/harshitrawat/summer/formation_energy/cifs"
compounds = {
    "mp-841.cif": "Li2O2",
    "mp-1960.cif": "Li2O",
    "mp-942733.cif": "Li7La3Zr2O12",
    "mp-2858.cif": "ZrO2",
    "mp-1968.cif": "La2O3",
}

# ---- 4. Predict formation energy per atom -----------------------------
for fname, label in compounds.items():
    struct = Structure.from_file(os.path.join(cif_dir, fname))
    comp = struct.composition
    n_atoms = comp.num_atoms

    # Convert to ASE
    ase_atoms = AseAtomsAdaptor.get_atoms(struct)

    # Assign calculator and predict energy
    ase_atoms.calc = calculator
    energy_total = ase_atoms.get_potential_energy()  # eV (total)

    # Reference energy from MACE chemical potentials
    ref_total = sum(comp[el] * mu_mace[el.symbol] for el in comp.elements)

    # Formation energy per atom
    e_form = (energy_total - ref_total) / n_atoms

    print(f"{label:15s}:  E_form (MACE_T2_w2_it3) = {e_form: .6f} eV/atom")

/home/phanim/harshitrawat/mace/mace/calculators/mace.py:166: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f=model_path, map_location=device)


Using head Default out of ['pt_head', 'Default']
No dtype selected, switching to float64 to match model dtype.
Li2O2          :  E_form (MACE_T2_w2_it3) = -0.311494 eV/atom


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 12 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


Li2O           :  E_form (MACE_T2_w2_it3) = -0.354533 eV/atom
Li7La3Zr2O12   :  E_form (MACE_T2_w2_it3) = -1.119615 eV/atom
ZrO2           :  E_form (MACE_T2_w2_it3) = -3.445901 eV/atom


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/pymatgen/core/structure.py:3107: UserWarning: Issues encountered while parsing CIF: 8 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  struct = parser.parse_structures(primitive=primitive)[0]


La2O3          :  E_form (MACE_T2_w2_it3) = -3.348167 eV/atom


In [2]:
from mp_api.client import MPRester
from pymatgen.core import Structure
from ase.io import write
from pymatgen.io.ase import AseAtomsAdaptor
import os

API_KEY = "j3J85pX4nLw6asHG9E2lbbCHEKDKgrjc"   # <-- put your Materials Project API key here
adaptor = AseAtomsAdaptor()

# Output directory
os.makedirs("mp_li_la_zr_o_cifs", exist_ok=True)

from itertools import combinations

wanted_elements = ["Li", "La", "Zr", "O"]
all_structures = []

with MPRester(API_KEY) as mpr:
    for r in [2, 3, 4]:  # binaries, ternaries, quaternaries
        for combo in combinations(wanted_elements, r):
            chemsys = "-".join(combo)
            docs = mpr.summary.search(
                chemsys=chemsys,
                fields=["material_id", "formula_pretty", "structure", "energy"]
            )
            print(f"{chemsys}: {len(docs)} entries")

            for doc in docs:
                struct = doc.structure
                ase_atoms = adaptor.get_atoms(struct)
                ase_atoms.info["energy"] = doc.energy
                all_structures.append(ase_atoms)

                # Save CIF
                cif_name = f"mp_li_la_zr_o_cifs/{doc.material_id}_{doc.formula_pretty}.cif"
                struct.to(fmt="cif", filename=cif_name)

# Write replay dataset
write("replay_li_la_zr_o.xyz", all_structures)
print(f"Saved {len(all_structures)} structures into replay_li_la_zr_o.xyz")


/home/phanim/harshitrawat/miniconda3/envs/mace_0.3.8/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_2750347/3549858856.py:22: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  docs = mpr.summary.search(


MPRestError: invalid fields requested: ['energy']. Available fields: ['builder_meta', 'nsites', 'elements', 'nelements', 'composition', 'composition_reduced', 'formula_pretty', 'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic', 'symmetry', 'property_name', 'material_id', 'deprecated', 'deprecation_reasons', 'last_updated', 'origins', 'warnings', 'structure', 'task_ids', 'uncorrected_energy_per_atom', 'energy_per_atom', 'formation_energy_per_atom', 'energy_above_hull', 'is_stable', 'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas', 'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct', 'is_metal', 'es_source_calc_id', 'bandstructure', 'dos', 'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering', 'total_magnetization', 'total_magnetization_normalized_vol', 'total_magnetization_normalized_formula_units', 'num_magnetic_sites', 'num_unique_magnetic_sites', 'types_of_magnetic_species', 'bulk_modulus', 'shear_modulus', 'universal_anis

In [5]:
from mp_api.client import MPRester
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import write
from itertools import combinations
import os

API_KEY = "j3J85pX4nLw6asHG9E2lbbCHEKDKgrjc"
adaptor = AseAtomsAdaptor()

os.makedirs("mp_li_la_zr_o_cifs", exist_ok=True)
wanted_elements = ["Li", "La", "Zr", "O"]

structures = []

with MPRester(API_KEY) as mpr:
    for r in [2, 3, 4]:  # binaries, ternaries, quaternaries
        for combo in combinations(wanted_elements, r):
            chemsys = "-".join(combo)
            docs = mpr.summary.search(
                chemsys=chemsys,
                fields=["material_id", "formula_pretty", "structure",
                        "nsites", "energy_per_atom"]
            )
            print(f"{chemsys}: {len(docs)} entries")

            for doc in docs:
                struct = doc.structure
                ase_atoms = adaptor.get_atoms(struct)

                # Convert per-atom energy to total energy
                total_energy = doc.energy_per_atom * doc.nsites
                ase_atoms.info["energy"] = total_energy

                structures.append(ase_atoms)

                # Save CIF
                cif_name = f"mp_li_la_zr_o_cifs/{doc.material_id}_{doc.formula_pretty}.cif"
                struct.to(fmt="cif", filename=cif_name)

# Save replay dataset
write("replay_li_la_zr_o.xyz", structures)
print(f"Saved {len(structures)} structures into replay_li_la_zr_o.xyz")


/tmp/ipykernel_2750347/4090857286.py:19: DeprecationWarning: Accessing summary data through MPRester.summary is deprecated. Please use MPRester.materials.summary instead.
  docs = mpr.summary.search(
Retrieving SummaryDoc documents: 100%|████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 61082.10it/s]


Li-La: 3 entries


Retrieving SummaryDoc documents: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 5526.09it/s]


Li-Zr: 1 entries


Retrieving SummaryDoc documents: 100%|█████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 202241.07it/s]


...

Li-Zr-O: 16 entries


Retrieving SummaryDoc documents: 100%|████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 61230.72it/s]


La-Zr-O: 6 entries


Retrieving SummaryDoc documents: 100%|████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 14768.68it/s]

Li-La-Zr-O: 3 entries
Saved 104 structures into replay_li_la_zr_o.xyz


In [3]:
!pip show mp-api


Name: mp-api
Version: 0.45.8
Summary: API Client for the Materials Project
Home-page: 
Author: 
Author-email: The Materials Project <feedback@materialsproject.org>
License: modified BSD
Location: /home/phanim/harshitrawat/miniconda3/lib/python3.13/site-packages
Requires: emmet-core, maggma, monty, msgpack, pymatgen, requests, setuptools, smart_open, typing-extensions
Required-by: 
